In [27]:
import matplotlib.pyplot as plt
import networkx as nx
import igraph as ig
import pandas as pd
import leidenalg as la
optimiser = la.Optimiser()

In [4]:
df = pd.read_csv('Data/NW_data/werkschool_nw_b=0.csv')
G = nx.from_pandas_edgelist(df, 'source_id', 'destination_id' ,create_using=nx.DiGraph())

In [16]:
tuples = [tuple([a, b]) for a, b in zip(df['source_id'], df['destination_id'])]
# Gm = igraph.Graph.TupleList(tuples, directed = True, edge_attrs = ['weight'])

In [17]:
tuples

[(92788, 85677),
 (91943, 75887),
 (93127, 61355),
 (93070, 89158),
 (92411, 70690),
 (91907, 63000),
 (92289, 67797),
 (91794, 76565),
 (92598, 82755),
 (91787, 80659),
 (92363, 78640),
 (91732, 87965),
 (91716, 64725),
 (92751, 79411),
 (93166, 83707),
 (91945, 71609),
 (92918, 60712),
 (92682, 89094),
 (93012, 66053),
 (92500, 85868),
 (92120, 68063),
 (91381, 90190),
 (91618, 75105),
 (92404, 61866),
 (91380, 70191),
 (92588, 77427),
 (92961, 71444),
 (91425, 88873),
 (92600, 69707),
 (91914, 77460),
 (91555, 86370),
 (92129, 76904),
 (91729, 70099),
 (92654, 90643),
 (92013, 63127),
 (92319, 89949),
 (92544, 68616),
 (91403, 70665),
 (92102, 74502),
 (91893, 66250),
 (92364, 66425),
 (92236, 62202),
 (92162, 69334),
 (91551, 87187),
 (92789, 69314),
 (93018, 90214),
 (92545, 85170),
 (91878, 77664),
 (93089, 75789),
 (92553, 77933),
 (91336, 68387),
 (92424, 88270),
 (91887, 78819),
 (91624, 91270),
 (92663, 69496),
 (91611, 68010),
 (93088, 78742),
 (91969, 62971),
 (92826, 87404

In [18]:
Gm = ig.Graph.TupleList(tuples, directed = True)


In [19]:
Gm.get_edgelist()


[(0, 1),
 (2, 3),
 (4, 5),
 (6, 7),
 (8, 9),
 (10, 11),
 (12, 13),
 (14, 15),
 (16, 17),
 (18, 19),
 (20, 21),
 (22, 23),
 (24, 25),
 (26, 27),
 (28, 29),
 (30, 31),
 (32, 33),
 (34, 35),
 (36, 37),
 (38, 39),
 (40, 41),
 (42, 43),
 (44, 45),
 (46, 47),
 (48, 49),
 (50, 51),
 (52, 53),
 (54, 55),
 (56, 57),
 (58, 59),
 (60, 61),
 (62, 63),
 (64, 65),
 (66, 67),
 (68, 69),
 (70, 71),
 (72, 73),
 (74, 75),
 (76, 77),
 (78, 79),
 (80, 81),
 (82, 83),
 (84, 85),
 (86, 87),
 (88, 89),
 (90, 91),
 (92, 93),
 (94, 95),
 (96, 97),
 (98, 99),
 (100, 101),
 (102, 103),
 (104, 105),
 (106, 107),
 (108, 109),
 (110, 111),
 (112, 113),
 (114, 115),
 (116, 117),
 (118, 119),
 (120, 121),
 (122, 123),
 (124, 125),
 (120, 126),
 (127, 128),
 (129, 130),
 (131, 132),
 (133, 134),
 (135, 136),
 (137, 138),
 (139, 140),
 (141, 142),
 (143, 144),
 (145, 146),
 (147, 148),
 (149, 150),
 (151, 152),
 (153, 154),
 (155, 156),
 (157, 158),
 (159, 160),
 (161, 162),
 (163, 164),
 (165, 166),
 (167, 168),
 (165

In [24]:
partition = la.find_partition(Gm, la.ModularityVertexPartition)

In [26]:
list(partition)

for i in list(partition):
    print(len(i))

629817
165314
1573
25
17
15
13
12
12
12
11
11
10
10
10
10
10
9
9
9
9
8
8
8
8
8
8
8
8
8
8
8
8
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3

In [28]:

diff = optimiser.optimise_partition(partition)